In [1]:
# pip install openai

In [2]:
import pandas as pd
import numpy as np
import openai
import nltk
from nltk.stem import WordNetLemmatizer

In [3]:
API_KEY = open("api_key.txt", "r").read()
openai.api_key = API_KEY

In [4]:
df = pd.read_csv('clean_data.csv')

In [5]:
# Extract first word(s) from the 'Text' column
df['First_Word'] = df['Food and Serving'].str.split().str[0]

food_items = df['First_Word'].values
# Remove the comma from each element in the array
food_items = [element.replace(',', '') for element in food_items]
# Convert all elements to lowercase
food_items = [element.lower() for element in food_items]


In [6]:

def convert_to_base_form(sentence):
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(sentence)
    base_words = [lemmatizer.lemmatize(word) for word in words]
    base_sentence = ' '.join(base_words)
    return base_sentence

# user_sentence = input("Enter a sentence: ")
# base_form = convert_to_base_form(user_sentence)
# print(base_form)


In [7]:
def find_word_index(sentence, word_array):
    sentence_words = sentence.lower().split()
    for i, word in enumerate(word_array):
        if word.lower() in sentence_words:
            return i
    return -1  # Return -1 if no match is found

# sentence = input("Enter a sentence: ")
# base_form = convert_to_base_form(sentence)
# word_index = find_word_index(base_form, food_items)
# print("Matched word index:", word_index)

# Displaying the row:

# def display_row_info(df, index):
#     if index != -1:
#         row = df.loc[index]
#         for column_name, element in row.iteritems():
#             print(f"{column_name}: {element}")
#     else:
#         print("Not registered")

# display_row_info(df, word_index)

def retrieve_row_info(df, index):
    row_info = []
    if index != -1:
        for column in df.columns:
            value = df.loc[index, column]
            row_info.append(f"{column}: {value}")
        return '\n'.join(row_info)
    else:
        return "None"

In [8]:
def get_column_names(df):
    return ', '.join(df.columns)

# Example usage
column_names = get_column_names(df)

In [9]:
chat_log= []

while True:
    sentence = input("User: ")
    base_form = convert_to_base_form(sentence)
    word_index = find_word_index(base_form, food_items)
    info = retrieve_row_info(df, word_index)
    if info != 'None':
        user_message = sentence + " give some information about the food item mentioned here like two or three lines and also display these values in tabluar format: '"+info+"'"
    else:
        user_message = sentence
    if sentence.lower() == "quit":
        print("Thank you for your time.")
        break
    else:
        chat_log.append({"role": "user", "content": user_message})
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=chat_log
        )
        assistant_response = response['choices'][0]['message']['content']
        print("ChatGPT: ", assistant_response.strip("\n").strip())
        chat_log.append({"role": "assistant", "content": assistant_response.strip("\n").strip()})
        

User: hello
ChatGPT:  Hello! How can I help you today?
User: give me some information about peaches
ChatGPT:  Peaches are a delicious fruit that are low in calories and fat. They are a good source of dietary fiber, potassium, and vitamin C. Here is the information you requested displayed in tabular format:

| Nutrient             | Amount        | % Daily Value |
|----------------------|--------------:|--------------:|
| Calories             | 60.0          | -             |
| Calories from Fat    | 0.0           | -             |
| Total Fat            | 0.5g          | 1.0%          |
| Sodium               | 0.0mg         | 0.0%          |
| Potassium            | 230.0mg       | 7.0%          |
| Total Carbohydrate   | 15.0g         | 5.0%          |
| Dietary Fiber        | 2.0g          | 8.0%          |
| Sugars               | 13.0g         | -             |
| Protein              | 1.0g          | -             |
| Vitamin A            | 6.0%          | -             |
| Vitam